PPO vs TD3

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import torch
import torch.nn as nn
import gym
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

# Adaptive Normalization
window_size = 126  
rolling_means = returns.rolling(window=window_size, min_periods=1).mean()
rolling_stds = returns.rolling(window=window_size, min_periods=1).std()
rolling_stds.replace(0, np.nan, inplace=True)
adaptive_returns = (returns - rolling_means) / rolling_stds
adaptive_returns.dropna(inplace=True)

# Ensure 'returns' and 'adaptive_returns' have matching indexes
aligned_returns = returns.loc[adaptive_returns.index].copy()

# Apply PCA to reduced adaptive returns
pca = PCA(n_components=4)
reduced_features = pca.fit_transform(adaptive_returns)

# K-Means Clustering
n_clusters = 4  
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
aligned_returns['Cluster'] = kmeans.fit_predict(reduced_features)

# One-Hot Encode Clusters
ohe = OneHotEncoder(sparse=False)
cluster_features = ohe.fit_transform(aligned_returns[['Cluster']])
cluster_features_df = pd.DataFrame(cluster_features, columns=[f'Cluster_{i}' for i in range(n_clusters)], index=aligned_returns.index)

# Merge into final dataset
returns = pd.concat([aligned_returns.drop(columns=['Cluster']), cluster_features_df], axis=1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
import gym
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import DummyVecEnv

# Custom Portfolio Environment
class PortfolioEnv(gym.Env):
    def __init__(self, returns):
        super(PortfolioEnv, self).__init__()
        self.returns = returns
        self.n_assets = returns.shape[1] - (n_clusters + 1)  # Excluding cluster labels and one-hot features
        self.observation_dim = self.n_assets + n_clusters  # Ensure correct shape
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(self.observation_dim,), dtype=np.float32)
        self.current_step = 0
    
    def reset(self):
        self.current_step = 0
        obs = np.append(
            self.returns.iloc[self.current_step, :-n_clusters-1].values,
            self.returns.iloc[self.current_step, -n_clusters:].values
        )
        return obs[:self.observation_dim]  
    
    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.returns) - 1
        
        portfolio_return = np.dot(action, self.returns.iloc[self.current_step, :-n_clusters-1].values)
        volatility = np.std(np.dot(self.returns.iloc[:self.current_step+1, :-n_clusters-1], action))
        var_threshold = np.percentile(portfolio_return, 5)
        cvar = np.mean(portfolio_return[portfolio_return <= var_threshold])
        r2r = portfolio_return / (volatility + 1e-6)
        
        reward =  abs(cvar) + r2r
        obs = np.append(
            self.returns.iloc[self.current_step, :-n_clusters-1].values,
            self.returns.iloc[self.current_step, -n_clusters:].values
        )
        
        return obs[:self.observation_dim], reward, done, {"r2r": r2r, "cvar": cvar, "portfolio_return": portfolio_return}
    
# Create Environment
env = DummyVecEnv([lambda: PortfolioEnv(returns)])

# Train PPO Model
ppo_model = PPO("MlpPolicy", env, verbose=1)
ppo_model.learn(total_timesteps=100000)

# Train TD3 Model
td3_model = TD3("MlpPolicy", env, verbose=1)
td3_model.learn(total_timesteps=100000)

# Evaluate Performance with clipped rewards
def evaluate_model(model, env, print_interval=100, reward_clip_range=(-1, 1)):
    obs = env.reset()
    done, rewards, cumulative_returns = False, [], []
    total_return = 1
    metrics = {"r2r": [], "cvar": [], "portfolio_return": []}
    
    step_counter = 0
    
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        
        # Clip rewards to avoid overflow
        reward = np.clip(reward, reward_clip_range[0], reward_clip_range[1])
        
        rewards.append(reward)
        total_return *= (1 + reward)
        
        # Prevent total_return from reaching infinity
        if np.isnan(total_return) or np.isinf(total_return):
            total_return = np.clip(total_return, -1e10, 1e10)
        
        cumulative_returns.append(total_return)
        
        info = info[0]  
        metrics["r2r"].append(info["r2r"])
        metrics["cvar"].append(info["cvar"])
        metrics["portfolio_return"].append(info["portfolio_return"])

        #step_counter += 1
        
    return np.mean(rewards), cumulative_returns, metrics

ppo_reward, ppo_cum_returns, ppo_metrics = evaluate_model(ppo_model, env)
td3_reward, td3_cum_returns, td3_metrics = evaluate_model(td3_model, env)

print(f"PPO Metrics - R2R: {np.mean(ppo_metrics['r2r'])}, CVaR: {np.mean(ppo_metrics['cvar'])}")
print(f"TD3 Metrics -  R2R: {np.mean(td3_metrics['r2r'])}, CVaR: {np.mean(td3_metrics['cvar'])}")


In [ ]:
# Convert portfolio returns to Pandas Series
ppo_portfolio_return = pd.Series(ppo_metrics["portfolio_return"])
td3_portfolio_return = pd.Series(td3_metrics["portfolio_return"])

# Compute rolling volatility (standard deviation over a 6-month window)
ppo_rolling_vol = ppo_portfolio_return.rolling(window=126).std()
td3_rolling_vol = td3_portfolio_return.rolling(window=126).std()

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(dates, ppo_rolling_vol, label="PPO Rolling Volatility", color="blue")
plt.plot(dates, td3_rolling_vol, label="TD3 Rolling Volatility", color="red")

plt.xlabel("Date")
plt.ylabel("Volatility (Rolling 6 Months)")
plt.title("Rolling Volatility Over Time (6-Month Window)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


# Summarize portfolio return contributions
ppo_total_return = sum(ppo_metrics["portfolio_return"])
td3_total_return = sum(td3_metrics["portfolio_return"])

# Pie chart labels
labels = ["PPO Portfolio Return", "TD3 Portfolio Return"]
returns = [ppo_total_return, td3_total_return]

# Define colors
colors = ["blue", "red"]

# Create Pie Chart
plt.figure(figsize=(8, 6))
plt.pie(returns, labels=labels, autopct="%1.1f%%", colors=colors, startangle=140, explode=[0.05, 0])
plt.title("Portfolio Return Contribution (PPO vs TD3)")
plt.show()



PPO-LSTM

In [ ]:
# === Custom Portfolio Gym Environment ===
class PortfolioEnv(gym.Env):
    def __init__(self, returns, n_clusters):
        super(PortfolioEnv, self).__init__()
        self.returns = returns
        self.n_assets = len(price_cols)
        self.n_clusters = n_clusters
        self.action_space = gym.spaces.Box(low=0, high=1, shape=(self.n_assets,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(self.n_assets + self.n_clusters,), dtype=np.float32)
        self.current_step = 0
        self.initial_capital = 1.0

    def reset(self):
        self.current_step = 0
        self.capital = self.initial_capital
        obs = np.append(self.returns.iloc[self.current_step, :-self.n_clusters].values, 
                        self.returns.iloc[self.current_step, -self.n_clusters:].values)

        # Ensure correct observation shape
        obs = self._fix_observation_shape(obs)
        return np.nan_to_num(obs)

    def step(self, action):
        self.current_step += 1
        done = self.current_step >= len(self.returns) - 1

        action = np.nan_to_num(action)
        action = np.exp(action) / np.sum(np.exp(action))  # Softmax normalization
        
        portfolio_return = np.dot(action, self.returns.iloc[self.current_step, :-self.n_clusters].values)
        self.capital *= (1 + portfolio_return)

        rolling_window = min(126, self.current_step + 1)
        volatility = np.std(np.dot(self.returns.iloc[self.current_step-rolling_window+1:self.current_step+1, :-self.n_clusters], action))
        volatility = max(volatility, 1e-6)
        
        past_returns = np.dot(self.returns.iloc[:self.current_step+1, :-self.n_clusters], action)
        var_threshold = np.percentile(past_returns, 5)
        cvar = np.mean(past_returns[past_returns <= var_threshold]) if np.any(past_returns <= var_threshold) else 0
        
        alpha, beta = 0.3, 0.2
        r2r = portfolio_return / volatility
        drawdown_penalty = abs(min(0, portfolio_return))
        reward = r2r - alpha * abs(cvar) - beta * drawdown_penalty

        obs = np.append(self.returns.iloc[self.current_step, :-self.n_clusters].values, 
                        self.returns.iloc[self.current_step, -self.n_clusters:].values)

        # Ensure correct observation shape
        obs = self._fix_observation_shape(obs)
        return np.nan_to_num(obs), reward, done, {"capital": self.capital, "r2r": r2r, "cvar": cvar}

    def _fix_observation_shape(self, obs):
        expected_shape = self.n_assets + self.n_clusters
        if obs.shape[0] > expected_shape:
            obs = obs[:expected_shape]  # Truncate if extra
        elif obs.shape[0] < expected_shape:
            obs = np.pad(obs, (0, expected_shape - obs.shape[0]), 'constant')  # Pad if missing
        return obs

# Create Gym Environment
env = DummyVecEnv([lambda: PortfolioEnv(returns, n_clusters)])

# === Custom LSTM Feature Extractor ===
class LSTMFeatureExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space, features_dim=128):
        super(LSTMFeatureExtractor, self).__init__(observation_space, features_dim)
        input_dim = observation_space.shape[0]
        self.lstm = nn.LSTM(input_dim, features_dim, batch_first=True)
        self.flatten = nn.Flatten()

    def forward(self, observations):
        observations = observations.unsqueeze(1)
        lstm_out, _ = self.lstm(observations)
        lstm_out = self.flatten(lstm_out[:, -1, :])
        return lstm_out

# Train PPO-LSTM Model
ppo_lstm_model = PPO(
    "MlpPolicy", env,
    policy_kwargs={"features_extractor_class": LSTMFeatureExtractor},
    gamma=0.993, learning_rate=0.0002, batch_size=128, ent_coef=0.012, verbose=1
)
ppo_lstm_model.learn(total_timesteps=100000)

# Train Standard PPO Model (without LSTM)
ppo_model = PPO(
    "MlpPolicy", env,
    gamma=0.91, learning_rate=0.0002, batch_size=64, ent_coef=0.05, verbose=1
)
ppo_model.learn(total_timesteps=100000)

# Evaluate & Plot Results
def evaluate_model(model, env):
    obs = env.reset()
    done, cumulative_returns = False, []
    r2r_values, cvar_values = [], []
    dates = data['Dates'].iloc[returns.index]
    
    while not done:
        action, _ = model.predict(np.nan_to_num(obs))
        obs, reward, done, info = env.step(action)
        cumulative_returns.append(info[0]["capital"])
        r2r_values.append(info[0]["r2r"])
        cvar_values.append(info[0]["cvar"])
    
    return dates[:len(cumulative_returns)], cumulative_returns, r2r_values, cvar_values

# Evaluate PPO-LSTM
dates, ppo_lstm_cum_returns, ppo_lstm_r2r, ppo_lstm_cvar = evaluate_model(ppo_lstm_model, env)

# Evaluate PPO
dates, ppo_cum_returns, ppo_r2r, ppo_cvar = evaluate_model(ppo_model, env)

# Plot PPO, PPO-LSTM Results
plt.figure(figsize=(10, 5))
plt.plot(dates, ppo_lstm_cum_returns, label='PPO-LSTM Cumulative Return', color='blue')
plt.plot(dates, ppo_cum_returns, label='PPO Cumulative Return', color='green')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.title('PPO vs PPO-LSTM Portfolio Performance')
plt.legend()
plt.show()

# Print final performance metrics
print(f"PPO-LSTM R2R: {np.mean(ppo_lstm_r2r):.4f}, CVaR: {np.mean(ppo_lstm_cvar):.4f}")


In [ ]:
# Compute daily portfolio returns from cumulative returns
ppo_lstm_daily_returns = np.diff(ppo_lstm_cum_returns) / ppo_lstm_cum_returns[:-1]

# Compute daily volatility 
ppo_lstm_daily_volatility = np.std(ppo_lstm_daily_returns)

# Annualize volatility
ppo_lstm_annual_volatility = ppo_lstm_daily_volatility * np.sqrt(252)

print(f"PPO-LSTM Portfolio Annual Volatility: {ppo_lstm_annual_volatility:.4%}")


In [ ]:
# Define six-month rolling window (126 trading days)
rolling_window = 126  

# Convert cumulative returns to daily returns
ppo_lstm_daily_returns = np.diff(ppo_lstm_cum_returns) / ppo_lstm_cum_returns[:-1]
dates_rolling = dates[1:]  # Align with daily returns

# Compute six-month rolling volatility (annualized)
rolling_volatility = pd.Series(ppo_lstm_daily_returns).rolling(rolling_window).std() * np.sqrt(252)

# Compute six-month rolling return
rolling_return = pd.Series(ppo_lstm_cum_returns).pct_change(rolling_window)

# Plot Rolling Volatility & Rolling Return
fig, ax1 = plt.subplots(figsize=(12, 5))

# Rolling Volatility (Left Y-axis)
ax1.set_xlabel('Date')
ax1.set_ylabel('Rolling Volatility (Annualized)', color='blue')
ax1.plot(dates_rolling, rolling_volatility, label='Rolling Volatility', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Rolling Return (Right Y-axis)
ax2 = ax1.twinx()
ax2.set_ylabel('Rolling Return (6-Month)', color='red')
ax2.plot(dates, rolling_return, label='Rolling Return', color='red', linestyle='dashed')
ax2.tick_params(axis='y', labelcolor='red')

# Formatting
plt.title('Six-Month Rolling Volatility & Rolling Return Over Time')
ax1.grid(True, linestyle='--', alpha=0.6)
fig.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import plotly.express as px

# Ensure 'Dates' column is properly formatted
data['Dates'] = pd.to_datetime(data['Dates'])
returns['Date'] = data.loc[returns.index, 'Dates']  # Aligning 'Date' with 'returns'

# Check if 'Cluster' exists in 'returns', if not, assign it from 'aligned_returns'
if 'Cluster' not in returns.columns:
    returns['Cluster'] = aligned_returns['Cluster']

# Ensure 'Cluster' is categorical
returns['Cluster'] = returns['Cluster'].astype('category')

# Cluster Distribution Over Time
plt.figure(figsize=(12, 6))
sns.histplot(data=returns, x='Date', hue='Cluster', bins=50, palette='tab10', multiple='stack')
plt.xlabel("Date")
plt.ylabel("Cluster Count")
plt.title("Cluster Distribution Over Time")
plt.xticks(rotation=45)
plt.show()

# Smoothed Cluster Distribution 
cluster_counts = returns.groupby(['Date', 'Cluster']).size().unstack(fill_value=0)
cluster_frequencies = cluster_counts.div(cluster_counts.sum(axis=1), axis=0).rolling(30).mean()

plt.figure(figsize=(12, 6))
cluster_frequencies.plot.area(stacked=True, alpha=0.7, cmap='tab10', figsize=(12, 6))
plt.xlabel("Date")
plt.ylabel("Cluster Frequency")
plt.title("Cluster Distribution Over Time (Smoothed)")
plt.xticks(rotation=45)
plt.legend(title="Clusters")
plt.show()

# Interactive Plot with Plotly
fig = px.area(cluster_frequencies, x=cluster_frequencies.index, y=cluster_frequencies.columns, 
              title="Interactive Cluster Distribution Over Time", labels={'value': 'Cluster Frequency'})
fig.update_layout(xaxis_title='Date', yaxis_title='Cluster Frequency')
fig.show()